# Multi panel model

Introduction:
------------------

Multi-panelModel: (One panel per segment, many orientations): In this model, only one panel per rooftop segment is modelled. It is based on detailed distributions information on the area, slope, and azimuth of all roof segments in a grid cell. Initially, PV power output for each rooftop segment is calculated separately considering the aspect, slope and area of the segment, but without considering shading and geometric placement issues in order to limit computational power requirements. The PV power output from all segments is totalled to return the idealized PV power for a grid cell. Finally, the output is reduced due to effects of geometric placement and shading (self-shading and object shading).

Methodology:
------------------
This model was developed using LiDAR and DSM data. Forty simulation areas have been generated in which 25 training areas are used to analyze the relationships between PV power production potential and urban forms.

Later parametrized equations, namely placement efficiency and shading efficiency are generated from the training areas, which were later evaluated using the evaluation areas. Placement efficiency was used to accurately calculate geometrically placed PV panels for an urban area without actually placing PV panels on rooftop building segments. Shading efficiency calculates the PV power output with shading losses instead of using the SEBE model. With the placement and shading efficiency, PV power output for an urban area can be calculated in a simple and fast manner.

Significance:
------------------
1. With fewer input parameters, can rapidly estimate the PV potential over an entire year.
2. Parametrized equations used in this model assist in calculating geometrically placed PV panels and account for shading losses.

Limitations:
-----------------
1. Assumes all rooftop building segments are equipped with PV panels neglecting safety, socio-economic and environmental factors.

Output:
-----------

Hourly PV power output [W/m2] for an urban area

# Prerequisite script

Attribute Table.ipynb

In [1]:
"""
Input Parameters
----------------
    l            : Length of PV panel in portait mode [m]
    w            : Width of PV panel in portait mode [m]
    pv_eff       : Efficiency of PV panel [%]
    alpha        : Temperature coefficient of PV panel [1/K]
    lamda_p      : Projected plan area fraction of building segments
    l_s          : Average building segment length [m] is the typical distance of the length of the roof, 
                   which tells you how far the shadows get on the roof and is calculated 
                   by taking the square root of the average segment area.
    f_f          : Fraction of flat building segments
    wire_losses  : Wiring losses
    f_c          : Capacity curve coefficient(f_c = 1 if all the rooftop segments are covered with panels)
    d            : Panel row spacing, can be optimized according to technical & Economical parameters
                   In this paper, d = 1.5 m
    β_o          : Optimal slope of PV panels on flat rooftops, can be optimized according to location
                   In this paper, β_0 = 53°(Freier et al., 2018) for the city of Berlin.
    albedo       : System albedo for an urban area
    segments     : Output from "Attribute Table.ipynb" which contains rooftop segments slope, aspect, geometry and segment ID.
    metadata     : Input meteorological data specifically formatted using 
                   UMEP -> Pre-processing -> Meteorological data -> Prepare existing data
                   Here, a sample Random Meteorological Day(RMD) for the month of June has been given as an example.
                   Annual weather data for the city of Berlin has been attached for reference.
    sol_angles   : Sun position angles for the urban area.
                   Here, solar angles for the RMD - June has been given as example

    Note: 1. In this script, l, w, pv_eff, alpha values has been taken for a standard PV module 
             but the user can change these parameters according to their module.
          2. lamda_p, l_s, f_f and urban_area values has been taken for an random urban area in the city of Berlin
          3. Input files and parameters can be found under the folder "Datasets" in Github
"""

import geopandas as gpd
import numpy as np
import pandas as pd
import math
import pvlib

l           = 1
w           = 1.6  
pv_eff      = 0.177 
alpha       = -0.0039  
lamda_p     = 0.398  
l_s         = 9.36  
f_f         = 0.413  
wire_losses = 0.8
f_c         = 1
d           = 1.5
β_o         = 40
albedo      = 0.2
segments    = gpd.read_file("segments.shp")
metdata     = np.loadtxt("june_RMD.txt", skiprows=1, dtype=float).reshape((-1, 24))
sol_angles  = pd.read_csv(f'june_angles_RMD.csv',index_col=0)

In [2]:
"""
Constants
---------
    ref_temp     : Reference PV cell temperature, default value = 25 °C
    k_t          : Reduction factor, default value = 0.05 °C/(W/m2)

        Reduction factor (kT) a constant defined as “expressing the changes in module performance due to 
        differences in the module’s actual , and nominal operating temperatures” (Ramirez Camargo et al., 2015). 
        A constant value for kT = 0.05 °C/(W/m2) was suggested by various authors (King et al., 2004), 
        (Ramirez Camargo et al., 2015) and also used in the PV-GIS web service for 
        building integrated systems.

        References:
        ----------

        Ramirez Camargo, L., Zink, R., Dorner, W., & Stoeglehner, G. (2015). 
        Spatio-temporal modelling of roof-top photovoltaic panels for improved technical potential assessment 
        and electricity peak load offsetting at the municipal scale. Computers, Environment, and Urban Systems, 
        52, 58–69. https://doi.org/10.1016/j.compenvurbsys.2015.03.002

        King, D. L., Boyson, W. E., & Kratochvill, J. A. (2004). Photovoltaic Array Performance Model. December.


"""

ref_temp = 25
k_t      = 0.05

In [3]:
"""
Theoretical PV panels
---------------------

    PV panels are calculated using rooftop building segment area without considering the geometry, tilt, 
    and aspect of the building segment.

"""
    
area_flat  = (math.cos(math.radians(β_o)) * w + d) * l
panel_area = l * w

for i in segments.index:
    
    if segments.loc[i, 'Slope'] == β_o:
        
        segments.loc[i, 'cal_panel'] = int(segments.loc[i, 'area'] / area_flat)
        
    else:
        
        segments.loc[i, 'cal_panel'] = int((segments.loc[i, 'area'] / math.cos(math.radians(segments.loc[i, 'Slope']))) / panel_area)

In [4]:
"""
Parametrized Equations
----------------------

    pl_eff       : Placement efficiency is the ratio of geometrically installed PV panels
                   to the theoretical PV panels.

    dir_eff      : Direct shading efficiency loss factor is the reduction of direct POA irradiance due to shading 
                   and is estimated by comparing the direct POA irradiance for Models 3 with building resolving model

    dif_eff      : Diffuse shading efficiency loss factor is the reduction of diffuse POA irradiance due to shading 
                   and is estimated by comparing the diffuse POA irradiance for Models 3 with building resolving model

    ref_eff      : Reflected shading efficiency loss factor is the reduction of diffuse POA irradiance due to shading 
                   and is estimated by comparing the reflected POA irradiance for Models 3 with building resolving model

"""
    
pl_eff  = (-0.0657 * f_f + 0.2736)
dir_eff = (-0.00041 * l_s - 0.00455)
dif_eff = (-0.00712 * l_s + 0.38399)
ref_eff = (0.01142 * l_s + 0.51025)

In [5]:
def multi_panel(slope, panels, aspect, index):

    for i in range(metdata.shape[0]):

        if metdata[[i], 14] > 0:

            Glob_hor = metdata[[i], 14]
            Diff_hor = metdata[[i], 21]
            Dir_hor = metdata[[i], 22]


            phi    = sol_angles.loc[int(metdata[[i], 2]), 'azi']
            gamma  = sol_angles.loc[int(metdata[[i], 2]), 'gamma']
            zenith = sol_angles.loc[int(metdata[[i], 2]), 'zen']
            aoi    = pvlib.irradiance.aoi(slope, aspect, zenith, phi)

            if gamma > 0 :

                lamda = math.cos(math.radians(phi - aspect)) + (math.tan(math.radians(gamma)) / math.tan(math.radians(slope)))

                if lamda <= 0 :

                    f_s1 = 1
                    f_s2 = 1

                else:

                    f_s1 = 0
                    x_l  = (math.cos(math.radians(slope)) + d/l) * math.sin(math.radians(phi - aspect)) / lamda
                    y_l  = 1 - ((math.cos(math.radians(slope)) + d/l) * math.tan(math.radians(gamma)) / (lamda * math.sin(math.radians(slope))))

                    if y_l <=0:

                        f_s2 = 0

                    elif abs(x_l) > (w / l):

                        f_s2 = 0

                    else:

                        f_s2 = (1 - (abs(x_l)/ (w / l))) * y_l

                f_sky1 = (1 + math.cos(math.radians(slope))) / 2

                B       = Dir_hor * (math.cos(math.radians(aoi)) / math.sin(math.radians(gamma)))                
                poa_dir = (1 - f_s1) * B * (1 - (dir_eff * gamma + 1))
                poa_dif = f_sky1 * Diff_hor * (1 - dif_eff)
                ref_dif = albedo * Glob_hor * (1 - f_sky1) * (1 - ref_eff)

                k =i+1

                segments.loc[index, f'{k}dir'] = poa_dir * pv_eff * (1 + alpha * ((metdata[[i], 11] + k_t * poa_dir) - ref_temp)) * panel_area * panels * wire_losses * f_c
                segments.loc[index, f'{k}dif'] = poa_dif * pv_eff * (1 + alpha * ((metdata[[i], 11] + k_t * poa_dif) - ref_temp)) * panel_area * panels * wire_losses * f_c
                segments.loc[index, f'{k}ref'] = ref_dif * pv_eff * (1 + alpha * ((metdata[[i], 11] + k_t * ref_dif) - ref_temp)) * panel_area * panels * wire_losses * f_c

    return segments




for i in segments.index:

    multi_panel(segments.loc[i, 'Slope'], segments.loc[i, 'cal_panel'] * (1 - pl_eff), segments.loc[i, 'Seg_Aspect'], i)


pv_power = pd.DataFrame()

for i in range(metdata.shape[0]):
    
    if metdata[[i], 14] > 0:
        
        k=i+1
        metdata1 = metdata[[i], :]
        hour = int(metdata1[0,2])
        pv_power.loc[k,f'Year']      = int(metdata1[0,0])
        pv_power.loc[k,f'Day']       = int(metdata1[0,1])
        pv_power.loc[k,f'Hour']      = hour
        pv_power.loc[k,'direct']    = segments.loc[: , f'{hour}dir'].sum(axis=0) / 1000000
        pv_power.loc[k,'diffuse']   = segments.loc[: , f'{hour}dif'].sum(axis=0) / 1000000
        pv_power.loc[k,'reflected'] = segments.loc[: , f'{hour}ref'].sum(axis=0) / 1000000
    
pv_power['Total_PVP'] = pv_power['direct'] + pv_power['diffuse'] + pv_power['reflected']

In [6]:
pv_power

,Year,Day,Hour,direct,diffuse,reflected,Total_PVP
5,2019.0,167.0,5.0,0.022738,0.171776,0.002357,0.196871
6,2019.0,171.0,6.0,0.202240,0.368100,0.008067,0.578407
7,2019.0,160.0,7.0,0.738752,0.620099,0.016205,1.375055
8,2019.0,152.0,8.0,0.785998,0.853868,0.018258,1.658124
9,2019.0,175.0,9.0,1.956214,0.732163,0.032250,2.720626
10,2019.0,179.0,10.0,0.270046,1.631835,0.017900,1.919781
11,2019.0,169.0,11.0,2.780177,0.891376,0.042009,3.713562
12,2019.0,174.0,12.0,3.106776,0.898540,0.044980,4.050296
13,2019.0,156.0,13.0,2.634451,1.116164,0.040698,3.791312
14,2019.0,165.0,14.0,2.943904,0.860092,0.042435,3.846431
